In [ ]:
etherpad_url = "http://localhost:9001/"
root_pad_title = "HierarchyRoot"
root_pad_renamed_title = "HierarchyRootRenamed"
child_pad_title = "HierarchyRoot/SubPad"
child_pad_renamed_title = "HierarchyRootRenamed/SubPad"
additional_child_pad_title = "HierarchyRootRenamed/SubPad2"
hash_tag_child = f"#{child_pad_title}"
hash_tag_child_renamed = f"#{child_pad_renamed_title}"
default_result_path = None
close_on_fail = False
transition_timeout = 10000

index_page = None
root_pad_url = None
child_pad_url = None
additional_child_pad_url = None

In [ ]:
import tempfile

work_dir = tempfile.mkdtemp()
if default_result_path is None:
    default_result_path = work_dir
work_dir

# ep_weave E2E Test - Hierarchical Pads

- Test data to prepare: Not required

In [ ]:
import importlib

import scripts.playwright
importlib.reload(scripts.playwright)

from scripts.playwright import *

await init_pw_context(close_on_fail=close_on_fail, last_path=default_result_path)

## Open etherpad_url and confirm the search box is editable.

In [ ]:
async def _step(page):
    await page.goto(etherpad_url)
    await expect(page.locator(".hashview-search-box")).to_be_editable()

    global index_page
    index_page = page

await run_pw(_step)

## Create HierarchyRoot from the search box.

In [ ]:
import re

async def _step(page):
    await page.locator(".hashview-search-box").fill(root_pad_title)
    await expect(page.locator('//button[text()="Create"]')).to_be_enabled()

    popup_future = page.wait_for_event('popup')
    await page.locator('//button[text()="Create"]').click()
    popup = await popup_future
    await expect(popup).to_have_title(re.compile(f"^{root_pad_title}"))
    await expect(popup.locator('//iframe[@name="ace_outer"]')).to_be_visible(timeout=transition_timeout)

    global root_pad_url
    root_pad_url = popup.url
    return popup

await run_pw(_step)

## Append the hierarchical hash tag to HierarchyRoot.

In [ ]:
import asyncio

async def extract_ace_inner_docbody(page):
    iframe_locator = page.frame_locator('iframe[name="ace_outer"]').frame_locator('iframe[name="ace_inner"]')
    await expect(iframe_locator.locator('#innerdocbody')).to_be_visible(timeout=transition_timeout)
    return iframe_locator.locator('#innerdocbody')

async def _step(page):
    inner_docbody = await extract_ace_inner_docbody(page)
    await expect(inner_docbody).to_be_editable(timeout=transition_timeout)
    await inner_docbody.focus()
    await inner_docbody.press('End')
    for _ in range(2):
        await inner_docbody.press('Enter')
    await inner_docbody.type(hash_tag_child, delay=100)
    await expect(inner_docbody).to_contain_text(hash_tag_child, timeout=transition_timeout)
    await page.wait_for_load_state('networkidle')
    await asyncio.sleep(1)  # wait for the flush to the server

await run_pw(_step)

## Verify the rollup shows the create link for the nested pad.

In [ ]:
async def _step(page):
    hash_create_link = page.locator('.hash-link.hash-create .hash-title a')
    await expect(hash_create_link).to_have_text(child_pad_title, timeout=transition_timeout)

await run_pw(_step)

## Open HierarchyRoot/SubPad from the rollup create link.

In [ ]:
async def _step(page):
    hash_create_link = page.locator('.hash-link.hash-create .hash-title a')
    await hash_create_link.click()
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{child_pad_title}"))

    global child_pad_url
    child_pad_url = page.url

await run_pw(_step)

## Reload HierarchyRoot and ensure the nested pad appears in the rollup.

In [ ]:
async def _step(page):
    await page.goto(root_pad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{root_pad_title}"))

    hash_link = page.locator('.hash-link:not(.hash-create) .hash-title a', has_text=child_pad_title)
    await expect(hash_link).to_have_count(2, timeout=transition_timeout)

    await page.goto(child_pad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{child_pad_title}"))

await run_pw(_step)

## Open HierarychyRoot from the breadcrumbs.

In [ ]:
async def _step(page):
    parent_link = page.locator(f'//a[contains(@class, "hashview-path-segment") and text()="{root_pad_title}"]')
    await expect(parent_link).to_be_visible(timeout=transition_timeout)

    await parent_link.click()
    await page.wait_for_load_state('networkidle')

await run_pw(_step)

## Rename HierarchyRoot to HierarchyRootRenamed.

In [ ]:
async def _step(page):    
    inner_docbody = await extract_ace_inner_docbody(page)
    await inner_docbody.focus()
    for _ in range(3):
        await inner_docbody.press('ArrowUp')
    await inner_docbody.press('End')
    for _ in range(len(root_pad_title) + 1):
        await inner_docbody.press('Backspace', delay=100)
    await inner_docbody.type(root_pad_renamed_title, delay=100)
    await inner_docbody.press('Enter')

    await expect(page).to_have_title(re.compile(f"^{root_pad_renamed_title}"), timeout=transition_timeout)

await run_pw(_step)

## Apply the change-title banner to update hashes.

In [ ]:
import asyncio

async def _step(page):
    change_title_button = page.locator('.hashview-change-title')
    await expect(change_title_button).to_be_visible(timeout=transition_timeout)
    await expect(change_title_button).to_contain_text(
        f'Title changed: {root_pad_renamed_title} from {root_pad_title}', timeout=transition_timeout
    )
    await change_title_button.click()
    await page.wait_for_load_state('networkidle')
    await asyncio.sleep(1)  # wait for the flush to the server

await run_pw(_step)

## Confirm HierarchyRoot now contains #HierarchyRootRenamed/SubPad.

In [ ]:
async def _step(page):
    await page.goto(root_pad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{root_pad_renamed_title}"))

    inner_docbody = await extract_ace_inner_docbody(page)
    await expect(inner_docbody).to_contain_text(hash_tag_child_renamed, timeout=transition_timeout)
    await expect(inner_docbody).not_to_contain_text(hash_tag_child)

    hash_link = page.locator('.hash-link .hash-title a', has_text=child_pad_renamed_title)
    await expect(hash_link).to_have_count(2, timeout=transition_timeout)

await run_pw(_step)

## Verify the child pad reflects the renamed hierarchy.

In [ ]:
async def _step(page):
    await page.goto(child_pad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{child_pad_renamed_title}"))

    inner_docbody = await extract_ace_inner_docbody(page)
    await expect(inner_docbody).to_contain_text(child_pad_renamed_title, timeout=transition_timeout)
    await expect(inner_docbody).not_to_contain_text(child_pad_title)

await run_pw(_step)

## Create HierarchyRootRenamed/SubPad2 from the index.

In [ ]:
async def _step(page):
    await page.goto(etherpad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page.locator('.hashview-search-box')).to_be_editable()

    await page.locator('.hashview-search-box').fill(additional_child_pad_title)
    await expect(page.locator('//button[text()="Create"]')).to_be_enabled()

    popup_future = page.wait_for_event('popup')
    await page.locator('//button[text()="Create"]').click()
    popup = await popup_future
    await popup.wait_for_load_state('networkidle')
    await expect(popup).to_have_title(re.compile(f"^{additional_child_pad_title}"))

    global additional_child_pad_url
    additional_child_pad_url = popup.url
    return popup

await run_pw(_step)

## Ensure HierarchyRootRenamed lists both child pads.

In [ ]:
async def _step(page):
    await page.goto(root_pad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{root_pad_renamed_title}"))

    # Child pages: 2 pages(SubPad and SubPad2) + Hash Link: 1 page(SubPad renamed)
    await expect(page.locator('.hash-link .hash-title a', has_text=child_pad_renamed_title)).to_have_count(3, timeout=transition_timeout)
    await expect(page.locator('.hash-link .hash-title a', has_text=additional_child_pad_title)).to_be_visible(timeout=transition_timeout)

    await page.goto(additional_child_pad_url)
    await page.wait_for_load_state('networkidle')
    await expect(page).to_have_title(re.compile(f"^{additional_child_pad_title}"))

await run_pw(_step)

Clean up

In [ ]:
await finish_pw_context()

In [ ]:
!rm -fr {work_dir}